# Rain prediction in Australia

**Task type:** Classification

**ML algorithm used:** Random Forest Classifier

**Metrics:** Accuracy, ROC-AUC

**Other methods used:** GridSearchCV, precision/recall balancing

<img src="../images/1786.webp" height=500 width=500>

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns 
import matplotlib.pyplot as plt
# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



In [2]:
import numpy as np
import pandas as pd
import seaborn as sns 

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline


In [3]:
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

# 1. Load data

In [4]:
df = pd.read_csv('../data/weatherAUS.csv')
df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


In [5]:
df.describe()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm
count,143975.000000,144199.000000,142199.000000,82670.000000,75625.000000,135197.000000,143693.000000,142398.000000,142806.000000,140953.000000,130395.00000,130432.000000,89572.000000,86102.000000,143693.000000,141851.00000
mean,12.194034,23.221348,2.360918,5.468232,7.611178,40.035230,14.043426,18.662657,68.880831,51.539116,1017.64994,1015.255889,4.447461,4.509930,16.990631,21.68339
std,6.398495,7.119049,8.478060,4.193704,3.785483,13.607062,8.915375,8.809800,19.029164,20.795902,7.10653,7.037414,2.887159,2.720357,6.488753,6.93665
min,-8.500000,-4.800000,0.000000,0.000000,0.000000,6.000000,0.000000,0.000000,0.000000,0.000000,980.50000,977.100000,0.000000,0.000000,-7.200000,-5.40000
25%,7.600000,17.900000,0.000000,2.600000,4.800000,31.000000,7.000000,13.000000,57.000000,37.000000,1012.90000,1010.400000,1.000000,2.000000,12.300000,16.60000
50%,12.000000,22.600000,0.000000,4.800000,8.400000,39.000000,13.000000,19.000000,70.000000,52.000000,1017.60000,1015.200000,5.000000,5.000000,16.700000,21.10000
75%,16.900000,28.200000,0.800000,7.400000,10.600000,48.000000,19.000000,24.000000,83.000000,66.000000,1022.40000,1020.000000,7.000000,7.000000,21.600000,26.40000
max,33.900000,48.100000,371.000000,145.000000,14.500000,135.000000,130.000000,87.000000,100.000000,100.000000,1041.00000,1039.600000,9.000000,9.000000,40.200000,46.70000


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145460 entries, 0 to 145459
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           145460 non-null  object 
 1   Location       145460 non-null  object 
 2   MinTemp        143975 non-null  float64
 3   MaxTemp        144199 non-null  float64
 4   Rainfall       142199 non-null  float64
 5   Evaporation    82670 non-null   float64
 6   Sunshine       75625 non-null   float64
 7   WindGustDir    135134 non-null  object 
 8   WindGustSpeed  135197 non-null  float64
 9   WindDir9am     134894 non-null  object 
 10  WindDir3pm     141232 non-null  object 
 11  WindSpeed9am   143693 non-null  float64
 12  WindSpeed3pm   142398 non-null  float64
 13  Humidity9am    142806 non-null  float64
 14  Humidity3pm    140953 non-null  float64
 15  Pressure9am    130395 non-null  float64
 16  Pressure3pm    130432 non-null  float64
 17  Cloud9am       89572 non-null

**Check how balanced the data is for the target column**

In [7]:
df.RainTomorrow.value_counts() 

No     110316
Yes     31877
Name: RainTomorrow, dtype: int64

In [13]:
df_No = df[df.RainTomorrow=='No'].sample(frac=1)
print("df_Nodf_No.shape


(31877, 23)

In [18]:
df_Yes = df[df.RainTomorrow=='Yes']
print("df_Yes shape =",df_Yes.shape)
df_Yes.head()

df_Yes shape = (31877, 23)


,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
8,2008-12-09,Albury,9.7,31.9,0.0,NaN,NaN,NNW,80.0,SE,...,42.0,9.0,1008.9,1003.6,NaN,NaN,18.3,30.2,No,Yes
10,2008-12-11,Albury,13.4,30.4,0.0,NaN,NaN,N,30.0,SSE,...,48.0,22.0,1011.8,1008.7,NaN,NaN,20.4,28.8,No,Yes
11,2008-12-12,Albury,15.9,21.7,2.2,NaN,NaN,NNE,31.0,NE,...,89.0,91.0,1010.5,1004.2,8.0,8.0,15.9,17.0,Yes,Yes
12,2008-12-13,Albury,15.9,18.6,15.6,NaN,NaN,W,61.0,NNW,...,76.0,93.0,994.3,993.0,8.0,8.0,17.4,15.8,Yes,Yes
16,2008-12-17,Albury,14.1,20.9,0.0,NaN,NaN,ENE,22.0,SSW,...,69.0,82.0,1012.2,1010.4,8.0,1.0,17.2,18.1,No,Yes


In [15]:
df_No = df_No.iloc[:df_Yes.shape[0],:]
df_No.shape

(31877, 23)

In [16]:
df_balanced = pd.concat([df_No, df_Yes])
df_balanced.shape
df = df_balanced

**Now we will deal with the null values**

# 2. Data preprocessing

In [12]:
zeros_cnt = df.isnull().sum().sort_values(ascending=False)
percent_zeros = (df.isnull().sum() / df.isnull().count()).sort_values(ascending=False)

missing_data = pd.concat([zeros_cnt, percent_zeros], axis=1, keys=['Total', 'Percent'])
missing_data
#missing_data.T

,Total,Percent
Sunshine,34080,0.534555
Evaporation,28855,0.452599
Cloud3pm,24714,0.387646
Cloud9am,24082,0.377733
Pressure3pm,8093,0.126941
Pressure9am,8089,0.126878
WindGustDir,5988,0.093924
WindGustSpeed,5963,0.093531
WindDir9am,5477,0.085908
WindDir3pm,2455,0.038507


**Let's drop those features where the missing/total coefficient is higher than 15%.**

In [ ]:
dropList = list(missing_data[missing_data['Percent'] > 0.15].index)
dropList
df.drop(dropList, axis=1, inplace=True)

In [ ]:
df['Location'].unique()

In [ ]:
#df.head()
df.shape

**A pairplot helps visualize dependencies and correlation between features. Some of them have quite obvious links.**

In [ ]:
sns.pairplot(df[:1000])

**Comments on the pairplots: There are few linear relations with high values of the r pearson coefficient.**

***
**Take the month out of the Date and move it to its own cell**

In [ ]:
df['Month'] = df.Date.apply(lambda x : int(x.split('-')[1]))

In [ ]:
df.drop(['Date'], axis=1, inplace=True) # drop the Date column
#df.drop(['Location'], axis=1, inplace=True)

In [ ]:
df.info()

**Let's encode categorical features using one-hot-encoding.**

In [ ]:
ohe = pd.get_dummies(data=df, columns=['WindGustDir','WindDir9am','WindDir3pm','Location'],drop_first=True)
ohe.info()

In [ ]:
from sklearn import preprocessing
from numpy import array

ohe['RainToday'] = df['RainToday'].astype(str)
ohe['RainTomorrow'] = df['RainTomorrow'].astype(str)

lb = preprocessing.LabelBinarizer()

ohe['RainToday'] = lb.fit_transform(ohe['RainToday'])
ohe['RainTomorrow'] = lb.fit_transform(ohe['RainTomorrow'])

**Drop missing values and create target column y and data X**

In [ ]:
ohe = ohe.dropna()
#ohe.drop('Location', axis=1, inplace=True)
y = ohe['RainTomorrow']
X = ohe.drop(['RainTomorrow'], axis=1)

# 3. Model building

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0) # split the data into train/test datasets
X_t,X_v,y_t,y_v = train_test_split(X_train, y_train, test_size=0.2,random_state=0) 
allValues = (X_t,X_v,y_t,y_v)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
X_train.info()

In [ ]:
def assessClassifier(clf,allValues):
    name = str(clf)
    X_train,X_test,y_train,y_test = allValues
    pipe = Pipeline([('scaler', StandardScaler()), (name, clf)])
    pipe.fit(X_train,y_train)
    y_pred = pipe.predict(X_test)
    f1 = f1_score(y_test, y_pred)
    precision = precision_score(y_test,y_pred)
    recall = recall_score(y_test,y_pred)
    return {'name':name,'f1':f1, 'precision':precision,'recall':recall}

In [ ]:
clf_Logistic = LogisticRegression()
clf_Forest = RandomForestClassifier()
clf_KNN = KNeighborsClassifier()
clf_SVC = SVC()
clf_XGBoost = XGBClassifier()
clf_NaiveB = GaussianNB()

In [ ]:
details = {'name':[],'f1':[],'precision':[],'recall':[]}
for clf in [clf_Logistic, clf_Forest, clf_KNN,clf_SVC, clf_XGBoost, clf_NaiveB]:#clf_Logistic, clf_Forest,
    results = assessClassifier(clf, allValues)
    for item in results.keys():
        details[item].append(results[item])

In [ ]:
df = pd.DataFrame(details)
df

***
**Testing grounds END**
***

***
***Support Vector Machines Tuneup***
***

In [ ]:
# Tunning Support Vector Machines
param_grid = {'C': [0.1, 1, 10, 100, 1000], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']} 
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3)
  
# fitting the model for grid search
grid.fit(X_t, y_t)


In [ ]:
# print best parameter after tuning
print(grid.best_params_)
  
# print how our model looks after hyper-parameter tuning
print(grid.best_estimator_)

In [ ]:
clf_SVC = SVC(C=1000, gamma=0.0001, kernel='rbf')
clf_SVC.fit(X_t,y_t)

In [ ]:
param_grid = { 
    'n_estimators': [100, 200],
    'max_features': ['auto'],
    'max_depth' : [4,5,8,10],
    'criterion' :['gini', 'entropy']
}


cv_RFC = GridSearchCV(estimator=RFC, param_grid=param_grid, cv=3)
cv_RFC.fit(X_train, y_train)

In [ ]:
cv_RFC.best_params_
#sorted(zip(cv_RFC.best_estimator_.feature_importances_,ohe.columns))

In [ ]:
pipe = Pipeline([('scaler', StandardScaler()), ('RFC', RandomForestClassifier(criterion='gini', 
                                                                              max_depth=10, 
                                                                              max_features='auto',
                                                                              n_estimators=200))])

In [ ]:
pipe = Pipeline([('scaler', StandardScaler()), ('Log', LogisticRegression())])

In [ ]:
pipe.fit(X_train, y_train)

# 4. Model evaluation

In [ ]:
pipe.score(X_train, y_train)

**Cross validation scores on the whole dataset:**

In [ ]:
from sklearn.model_selection import cross_val_score
cross_val_score(pipe, X, y, cv=5).mean()

In [ ]:
y_pred = pipe.predict(X_test)

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

#confusion_matrix(y_test, y_pred)
accuracy_score(y_test, y_pred)

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

#recall_score(y_test, y_pred)
#precision_score(y_test, y_pred)
f1_score(y_test, y_pred)

# 5. Plotting precision-recall & ROC curves.

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

ns_probs = [0 for _ in range(len(y_test))]
lr_probs = pipe.predict_proba(X_test)
lr_probs = lr_probs[:, 1]

ns_auc = roc_auc_score(y_test, ns_probs)
lr_auc = roc_auc_score(y_test, lr_probs)

print('No Skill: ROC AUC=%.3f' % (ns_auc))
print('RFC: ROC AUC=%.3f' % (lr_auc))

# calculate roc curves
ns_fpr, ns_tpr, _ = roc_curve(y_test, ns_probs)
lr_fpr, lr_tpr, _ = roc_curve(y_test, lr_probs)
# plot the roc curve for the model
plt.plot(ns_fpr, ns_tpr, linestyle='--', label='Dummy Classifer')
plt.plot(lr_fpr, lr_tpr, marker='.', label='RFC')
# axis labels
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
# show the legend
plt.legend()
# show the plot
plt.show()

**Let's plot a graph to identify the threshold influence on the scores**

In [ ]:
from sklearn.metrics import precision_recall_curve
y_scores = pipe.predict_proba(X_train)[:,1]
#y_scores

precisions, recalls, thresholds = precision_recall_curve(y_train, y_scores)

def plot_prc (precisions, recalls, thresholds):
    plt.plot(thresholds, precisions[:-1], 'b--', label='Precision')
    plt.plot(thresholds, recalls[:-1], 'g-', label='Recall')
    plt.xlabel('Thresholds')
    plt.legend(loc='center left')
    plt.ylim([0,1])

plot_prc(precisions, recalls, thresholds)

In [ ]:
#y_pred = clf.predict(X_test)  # default threshold is 0.5
y_pred1 = (pipe.predict_proba(X_train)[:,1] >= 0.8).astype(int) # set threshold as 0.3
precision_score(y_train, y_pred1)

**Here we can clearly see the balance between precision & recall. 
So if we want a higher recall, we can shift a threshold to a higher value.**

**However, you should decide on the threshold with a thorough analysis not to miss-out on the model performance later.**

# 6. Conclusion

**So, we have build a quite simple Random Forest Classifier using the features from dataset applying one-hot-encoding to the categorical features.**

**The accuracy-score for the out-of-the-box model is around 85% which is not bad. The AUC score is 0.862.**

**We have also conducted an experiment with shifting the decision boundary for the model which resulted in a precision score spike. This is the technique you can use to manually set the threshold for the trained classifier.**